**Several case studies from https://people.iee.ihu.gr/~vkostogl/en/Epixeirisiaki/Linear%20Programming_case%20studies_EN_29-5-2012.pdf**

Many thanks to Prof. Vassilis Kostoglou

# Case Study: Portfolio Selection
- Select portfolio package from set of investments
- Maximize expected return or minimize risk

## Data

Expected annual return rates of investments

In [1]:
investments = ["A", "B", "C", "D", "E", "Z"]
shares = ["A", "B", "C", "D"]
manufacturing = ["A", "B"]
food_beverage = ["C", "D"]
mutual_funds = ["E", "Z"]

ex_returns = {
    "A": 15.4,  # percent
    "B": 19.2,
    "C": 18.7,
    "D": 13.5,
    "E": 17.8,
    "Z": 16.3,
}

## Requirements
- Total money = €90000
- Amount in shares of a sector no larger than 50% of total available
- Amount in shares with the larger return of a sector less or equal to 80% of sector’s total amount
- Amount in manufacturing company Β less or equal to 10% of the whole share amount
- Amount in mutual funds less or equal to 25% of the amount in manufacturing shares

## Solution

### Decision variables
x_i: Amount invested in investment i

In [2]:
import pulp

prob = pulp.LpProblem("Portfolio_Selection", pulp.LpMaximize)

x1 = pulp.LpVariable("Amount1", 0, 90000, pulp.LpInteger)
x2 = pulp.LpVariable("Amount2", 0, 90000, pulp.LpInteger)
x3 = pulp.LpVariable("Amount3", 0, 90000, pulp.LpInteger)
x4 = pulp.LpVariable("Amount4", 0, 90000, pulp.LpInteger)
x5 = pulp.LpVariable("Amount5", 0, 90000, pulp.LpInteger)
x6 = pulp.LpVariable("Amount6", 0, 90000, pulp.LpInteger)

decision_vars = [x1, x2, x3, x4, x5, x6]

### Objective function
Maximize sum(x_i * return_i)

In [3]:
prob += (x1 * ex_returns["A"] / 100 + x2 * ex_returns["B"] / 100 + x3 * ex_returns["C"] / 100
    + x4 * ex_returns["D"] / 100 + x5 * ex_returns["E"] / 100 + x6 * ex_returns["Z"] / 100)

### Constraints

In [4]:
total_money = 90000
prob += x1 + x2 + x3 + x4 + x5 + x6 == 90000

# Amount in shares of a sector no larger than 50% of total available
prob += x1 + x2 <= 45000
prob += x3 + x4 <= 45000

# Amount in shares with the larger return of a sector less or equal to 80% of sector’s total amount
prob += x2 <= 0.80 * (x1 + x2) # Shares A and B
prob += x3 <= 0.80 * (x3 + x4) # Shares C and D

# Amount in manufacturing company Β less or equal to 10% of the whole share amount
prob += x2 <= 9000

# Amount in mutual funds less or equal to 25% of the amount in manufacturing shares
prob += x4 + x5 <= 0.25 * (x1 + x2)

### Problem solved

In [5]:
prob.solve()

print("Status:", pulp.LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)
    
print("Money earned:", pulp.value(prob.objective))

Status: Optimal
Amount1 = 27000.0
Amount2 = 9000.0
Amount3 = 36000.0
Amount4 = 9000.0
Amount5 = 0.0
Amount6 = 9000.0
Money earned: 15300.0


---

# Case Study: Advertising Media Selection
- Maximize total audience rate

## Data

For each time section, there's cost of one view and units of expected audience rate of one view

In [6]:
media = {
    "Fri-D": {"cost": 400, "audience": 5000},
    "Sat-D": {"cost": 450, "audience": 5500},
    "Sun-D": {"cost": 450, "audience": 5700},
    "Fri-N": {"cost": 500, "audience": 7500},
    "Sat-N": {"cost": 550, "audience": 8200},
    "Sun-N": {"cost": 550, "audience": 8400},
}

periods = ["Fri-D", "Sat-D", "Sun-D", "Fri-N", "Sat-N", "Sun-N"]
friday = ["Fri-D", "Fri-N"]
saturday = ["Sat-D", "Sat-N"]
sunday = ["Sun-D", "Sun-N"]
day = ["Fri-D", "Sat-D", "Sun-D"]
night = ["Fri-N", "Sat-N", "Sun-N"]

## Requirements
- Goal: Invest in different time zones to maximize the total audience rate
- Total money = €45000
- Max amount for Friday = €11000
- Max amount for Saturday = €14400
- Total day views: At least 20
- Total night views: At least 50% of the total
- Max views for each period: Each day 12, each night 18

## Solution

### Decision variables
x_i: Number of views bought on period i

In [7]:
prob2 = pulp.LpProblem("Advertising", pulp.LpMaximize)

views_fri_d = pulp.LpVariable("Views_Fri-D", 0, 12, pulp.LpInteger)
views_sat_d = pulp.LpVariable("Views_Sat-D", 0, 12, pulp.LpInteger)
views_sun_d = pulp.LpVariable("Views_Sun-D", 0, 12, pulp.LpInteger)
views_fri_n = pulp.LpVariable("Views_Fri-N", 0, 18, pulp.LpInteger)
views_sat_n = pulp.LpVariable("Views_Sat-N", 0, 18, pulp.LpInteger)
views_sun_n = pulp.LpVariable("Views_Sun-N", 0, 18, pulp.LpInteger)

decision_vars = [views_fri_d, views_sat_d, views_sun_d, views_fri_n, views_sat_n, views_sun_n]

### Objective function
Maximize sum(x_i * audience_i)

In [8]:
prob2 += ((views_fri_d * media["Fri-D"]["audience"]) + (views_sat_d * media["Sat-D"]["audience"]) +
        (views_sun_d * media["Sun-D"]["audience"]) + (views_fri_n * media["Fri-N"]["audience"]) +
        (views_sat_n * media["Sat-N"]["audience"]) + (views_sun_n * media["Sun-N"]["audience"]))

### Constraints

In [9]:
# Total money = €45000
total_money = 45000
prob2 += ((views_fri_d * media["Fri-D"]["cost"]) + (views_sat_d * media["Sat-D"]["cost"]) +
        (views_sun_d * media["Sun-D"]["cost"]) + (views_fri_n * media["Fri-N"]["cost"]) +
        (views_sat_n * media["Sat-N"]["cost"]) + (views_sun_n * media["Sun-N"]["cost"])) <= 45000

# Max amount for Friday = €11000
prob2 += views_fri_d * media["Fri-D"]["cost"] + views_fri_n * media["Fri-N"]["cost"] <= 11000

# Max amount for Saturday = €14400
prob2 += views_sat_d * media["Sat-D"]["cost"] + views_sat_n * media["Sat-N"]["cost"] <= 14400

# Total day views: At least 20
prob2 += views_fri_d + views_sat_d + views_sun_d >= 20

# Total night views: At least 50% of the total
prob2 += views_fri_n + views_sat_n + views_sun_n >= 0.50 * (views_fri_n + views_sat_n + views_sun_n + 
                                                     views_fri_d + views_sat_d + views_sun_d)

# Max views for each period: Each day 12, each night 18
# Defined in decision variables

### Problem solved

In [10]:
prob2.solve()

print("Status:", pulp.LpStatus[prob2.status])

money_spent = 0

for v in prob2.variables():
    print(v.name, "=", v.varValue)
    money_spent += v.varValue * media[v.name[-5:].replace("_", "-")]["cost"]
    
print("Total audience:", pulp.value(prob2.objective))
print("Money spent:", money_spent)

Status: Optimal
Views_Fri_D = 5.0
Views_Fri_N = 18.0
Views_Sat_D = 10.0
Views_Sat_N = 18.0
Views_Sun_D = 12.0
Views_Sun_N = 18.0
Total audience: 582200.0
Money spent: 40700.0


---

# Case Study: HR Management (Allocation)
- Recruit seasonal staff
- Allocate staff to shifts
- Minimize total cost

## Data and Requirements

Time periods, which shifts each of them belong to, min number of employees to be present at each period

Employee cost per day for each shift

In [11]:
shift_costs = {
    1: 230,
    2: 220,
    3: 225,
    4: 240,
    5: 260
}

periods = {
    "07-09": {"shifts": [1], "min_employees": 35},
    "09-11": {"shifts": [1, 2], "min_employees": 68},
    "11-13": {"shifts": [1, 2], "min_employees": 60},
    "13-15": {"shifts": [1, 2, 3], "min_employees": 57},
    "15-17": {"shifts": [2, 3, 4], "min_employees": 65},
    "17-19": {"shifts": [3, 4], "min_employees": 63},
    "19-21": {"shifts": [3, 4], "min_employees": 72},
    "21-23": {"shifts": [4], "min_employees": 33},
    "23-07": {"shifts": [5], "min_employees": 12},
}

## Solution

### Decision variables
x_i: Number of people working at shift i

In [12]:
prob3 = pulp.LpProblem("Allocation", pulp.LpMinimize)

x1 = pulp.LpVariable("People1", 0, None, pulp.LpInteger)
x2 = pulp.LpVariable("People2", 0, None, pulp.LpInteger)
x3 = pulp.LpVariable("People3", 0, None, pulp.LpInteger)
x4 = pulp.LpVariable("People4", 0, None, pulp.LpInteger)
x5 = pulp.LpVariable("People5", 0, None, pulp.LpInteger)

### Objective function
Minimize sum(x_i * cost_i)

In [13]:
prob3 += x1 * shift_costs[1] + x2 * shift_costs[2] + x3 * shift_costs[3] + x4 * shift_costs[4] + x5 * shift_costs[5]

### Constraints
Some redundant constraints are not written here

In [14]:
prob3 += x1 >= 35
prob3 += x1 + x2 >= 68
prob3 += x1 + x2 + x3 >= 57
prob3 += x2 + x3 + x4 >= 65
prob3 += x3 + x4 >= 72
prob3 += x4 >= 33
prob3 += x5 >= 12

### Problem solved

In [15]:
prob3.solve()

print("Status:", pulp.LpStatus[prob3.status])

for v in prob3.variables():
    print(v.name, "=", v.varValue)
    
print("Total cost:", pulp.value(prob3.objective))

Status: Optimal
People1 = 35.0
People2 = 33.0
People3 = 39.0
People4 = 33.0
People5 = 12.0
Total cost: 35125.0
